In [1]:
import bs4
import re
import xlrd
import hashlib
import openpyxl 

In [2]:
from urllib.request import urlopen as uReq

In [3]:
from bs4 import BeautifulSoup as soup

In [4]:
from openpyxl import *

In [5]:
wb_keywords = xlrd.open_workbook("Desktop\Files for Scraping\ключевые слова.xlsx")
sheet_keywords = wb_keywords.sheet_by_index(0)
sheet_keywords.cell_value(0, 0)
sheet_keywords.nrows
keywords = []
for i in range(sheet_keywords.nrows):
    keywords.append(sheet_keywords.cell_value(i, 0))
keywords[0], keywords[sheet_keywords.nrows-1]

('Международная неделя инвесторов в МФЦА 2020', 'centras group')

In [6]:
wb_website = xlrd.open_workbook("Desktop\Files for Scraping\Список сайтов для бота.xlsx")
sheet_website = wb_website.sheet_by_index(0)
sheet_website.cell_value(36, 1)
websites = []
for i in range(sheet_website.nrows):
    websites.append(sheet_website.cell_value(i,1))

In [43]:
#read old hashes
wb_hashes = xlrd.open_workbook("Desktop\Files for Scraping\Hashes.xlsx")
sheet_hashes = wb_hashes.sheet_by_index(0)
hashes = []
for i in range(sheet_hashes.nrows):
    hashes.append(sheet_hashes.cell_value(i,0))

In [44]:
hashes[0]

'4abec1ab88284ab4570baffa44b05197714131b34a28c5e77ad2f687'

In [45]:
counter = 0
for i in range(len(websites)):
    if(websites[i]=="http://www.audit.kz/" or websites[i]=="https://pnk.kz/" or websites[i]=="https://aifc.kz/ru/events/" or websites[i] == "https://www2.deloitte.com/kz/ru.html" or websites[i]== "https://www.apk-inform.com/ru"):
        continue
    uClient = uReq(websites[i])
    page_html = uClient.read()
    uClient.close()
    print("___________________")
    page_soup = soup(page_html, "html.parser")
    
    page_str = str(page_soup.body)
    begin = int(0.2 * len(page_str))
    end = int(0.8 * len(page_str))
    str_concat = page_str[begin:end]
    
    print(hashlib.sha224(str_concat.encode('utf-8')).hexdigest())
    if(hashes[counter] == hashlib.sha224(str_concat.encode('utf-8')).hexdigest()):
        counter = counter + 1
        continue
    else:
        #write a new hash to excel file
        hashes_wb = load_workbook("Desktop\Files for Scraping\Hashes.xlsx")
        hashes_ws = hashes_wb["Sheet1"]
        hashes_ws.cell(counter + 1, 1).value = hashlib.sha224(str_concat.encode('utf-8')).hexdigest()
        hashes_wb.save("Desktop\Files for Scraping\Hashes.xlsx")
        
        #parse the html file to find keywords
        print("Check this website: " + websites[i])
        for j in range(len(keywords)):
            temp = []
            temp = re.findall(keywords[j], str(page_soup.body))
            if(temp!= []):
                print("yes")
                print(str(temp[0]) + " " + str(len(temp)))
    counter = counter + 1

___________________
4abec1ab88284ab4570baffa44b05197714131b34a28c5e77ad2f687
___________________
3fe1261008385c2ba50ed9f877f6564e585af0528b60de156e1ed1ac
___________________
9753863938798280d44141a636807959eced5d63ea8d90754e56ef26
___________________
f0ecbb8dd8ac23efc7f3f565e48b33d53efc167e44b803df56f46302
___________________
baaa3bb85a9204af6a7a21f81c02c67fab5997f3f64ab3f7a2dcf3a5
___________________
84774c4acdd4f9bbf868c884d56680bf5736a569f224436b786bd6ea
___________________
92435491a8fa54360c853027506fd41f49307fa1c3fe3015ff2def3a
Check this website: http://www.afk.kz/ru/sobyitiya-afk/
___________________
4a7951694c1f79de4c45db4a3c08ae247a1fd80e7076936d73ac5770
Check this website: http://www.ank.kz/events
yes
conference 2
yes
forum 1
___________________
6794f628c4559bd39369f7c23953bb88f0bb0a95a47255036557370f
___________________
bc1d7b257d403e6c6514e6f60dd9d1e8afe6f93ce83316e5b727378c
___________________
10a5552c45739fa8577a818d527cd1699ac01124ee1ac33ea4b28487
___________________
4b3

In [19]:
counter

32